In [ ]:
# import modules and define functions
from pyscf import gto, scf
import pandas as pd
import numpy as np
from scipy import linalg
import os
from functools import reduce
os.environ['OMP_NUM_THREADS'] = "1"
def make_rdm1(mo_coeff, mo_occ):
    mocc = mo_coeff[:,mo_occ>0]
    return np.dot(mocc*mo_occ[mo_occ>0], mocc.conj().T)

# Mulliken charge population

In [ ]:
# build CO with minimal basis
atom = 'C 0 0 0; O 0 0 1.13'
basis = 'sto-3g'
charge = 0
spin = 0
co_0_1 = gto.Mole().build(atom=atom,basis=basis,charge=charge,spin=spin)
# calculate S
S = co_0_1.intor_symmetric('int1e_ovlp')


In [ ]:
# perform SCF 
co_0_1_rhf = scf.RHF(co_0_1).run()

In [ ]:
## calculate PS
# calculate P
mo_coeff = co_0_1_rhf.mo_coeff
mo_occ = co_0_1_rhf.mo_occ
dm = make_rdm1(mo_coeff,mo_occ)
# calculate PS
PS = np.dot(dm, S)
print('The PS matrix has the size of %s by %s.' %(PS.shape))

In [ ]:
## on which atom are the basis sets
# number of atomic orbitals
nao = co_0_1.nao
print('There are %s of atomic orbitals.' %(nao))
# number of basis functions (e.g. 2p orbitals are counted as 1)
num_bs = co_0_1.nbas
# find out on which atom is each atomic orbital
nao_atom_idx = np.zeros(nao)
count = 0
for bs_i in range(num_bs):
    for i in range(co_0_1.bas_angular(bs_i)*2+1):
        nao_atom_idx[count] = co_0_1.bas_atom(bs_i)
        count += 1


In [ ]:
# find tr(PS_\mu\mu), mu belongs to A and calculate q_A
num_atoms = co_0_1.natm
qs_nuc = [co_0_1.atom_charge(i) for i in range(num_atoms)]
asyms = []
qs_mulliken = []
print('The Mulliken charge for each atom in the molecule CO is:')
for i in range(num_atoms):
    ne_mulliken = np.trace(PS[nao_atom_idx == i].T[nao_atom_idx == i])
    q_mulliken = qs_nuc[i] - ne_mulliken
    asym = co_0_1.atom_symbol(i)
    print('%s: %.4f' %(asym, q_mulliken))

# Loewdin charge population

In [ ]:
# find tr(PS_\mu\mu), mu belongs to A and calculate q_A
PS_L = reduce(lambda a, b: np.dot(a,b), [linalg.sqrtm(S), dm, linalg.sqrtm(S)])
num_atoms = co_0_1.natm
qs_nuc = [co_0_1.atom_charge(i) for i in range(num_atoms)]
asyms = []
qs_mulliken = []
print('The Loewdin charge for each atom in the molecule CO is:')
for i in range(num_atoms):
    ne_mulliken = np.trace(PS_L[nao_atom_idx == i].T[nao_atom_idx == i])
    q_mulliken = qs_nuc[i] - ne_mulliken
    asym = co_0_1.atom_symbol(i)
    print('%s: %.4f' %(asym, q_mulliken))

# Charge population as we transform the atomic orbitals closer to orthonormal

In [ ]:
# find tr(PS_\mu\mu), mu belongs to A and calculate q_A
data_dict = {}
for n in np.linspace(0,1,11):
    PS_n = reduce(lambda a, b: np.dot(a,b), [linalg.fractional_matrix_power(S, n), dm, linalg.fractional_matrix_power(S, 1-n)])
    num_atoms = co_0_1.natm
    qs_nuc = [co_0_1.atom_charge(i) for i in range(num_atoms)]
    asyms = []
    qs_mulliken = []
    data = {}
    for i in range(num_atoms):
        ne_mulliken = np.trace(PS_n[nao_atom_idx == i].T[nao_atom_idx == i])
        q_i = qs_nuc[i] - ne_mulliken
        asym = co_0_1.atom_symbol(i)
        data[asym] = q_i
    data_dict[n] = data

In [ ]:
pd.DataFrame(data_dict)